# MBR FX

MBR Functions was a set of functions I created my first year at Capital One, that i further optimized as a member of first party fraud.

The powerpoint slides often contain hitsorical views of KPI but to deeply enhance understanding of performance, it helps to have an understanding of the Month over month variation. 

The following function take pandas dataframes that are oriented in a long-data format, and output the month-over-month percent variation in the metric, split by any segments or tactics of interest.


In [ ]:
import datetime
import dateutil.relativedelta
import calendar
import pandas as pd

today = datetime.date.today()


In [ ]:
def create_month_over_month_percents_fraud_losses(dataframe, metric = 'FRAUD_LOSSES'):
    ###create a function that calculates how the most recent month's losses compare to past
    ### months, subdivided by segment of interest
    ### the dataframe has to have key [CHARGEOFF_MONTH] when aggregating losses
    ### the dataframe has to have the either key [SEGMENT] or [TACTIC] to split losses by
    ### the parameter, metric is the specific value of interest, for this use case we will look at [FRAUD_LOSSES]
    
    #convert the text based date into a datetime so we can sort
    dataframe['CHARGEOFF_MONTH'] = pd.to_datetime(dataframe['CHARGEOFF_MONTH'], format = '%Y-%m-%d')
    
    #create a date index used to find months of interest
    last_month = today-dateutil.relativedelta.relativedelta(months=1).replace(day=1)
    two_months_ago = date - dateutil.relativedelta.relativedelta(months = 1)
    
    #create blank lists we can use to subdivide fraud losses by:
    segment_list = []
    tactic_list = []
    value_list = []
    
    #use exception handling to calculate the delta in losses by segment first, and if that fails, try seperating by tactic
    
    try:
        for segment in dataframe['SEGMENT'].unique():
            #pull the losses from the months of interest
            last_months_data = dataframe.loc[(dataframe['CHARGEOFF_MONTH'] == last_month) & (dataframe['SEGMENT'] == segment)][metric].reset_index()
            two_months_data = dataframe.loc[(dataframe['CHARGEOFF_MONTH'] == two_months_ago) & (dataframe['SEGMENT']==segment)][metric].reset_index()
            
            #calculate Month Over Month Change
            numerator = (two_months_data[metric] - last_months_data[metric]).iloc[0]
            denominator = two_months_data[metric].iloc[0]
            
            #prevent Div / 0 errors
            if denominator == 0:
                monthly_diff = 0.0
            else:
                monthly_diff = numerator/denominator
            #change the decimal to a percentage, rounded to two decimals
            monthly_diff *= 100
            
            try:
                monthly_diff = round(monthly_diff,2)
            except TypeError:
                monthly_diff = float(monthly_diff)
                monthly_diff = np.round(monthly_diff,2)
            
            #take the rounded number into a consistent string output, depending on a negative/positive monthly difference
            
            if str(monthly_diff)[0] == '-':
                monthly_diff = str(monthly_diff )[:5] + ' %'
            else:
                monthly_diff = str(monthly_diff)[:4] + ' %'
            
            #append that particular value into the list and iterate over all segments
            segment_list.append(segment)
            value_list.append(monthly_diff)
            
            #create a dataframe that combines the lists into a tabular, sensible format
            final_df = pd.DataFrame('Segment':segment_list)
            final_df['monthly_difference'] = pd.DataFrame(value_list)
            final_df = final_df.transpose()
            final_df.insert(0, 'Metric: ' + metric, ['Segment', 'Monthly % Difference'])
            return(final_df)
        
        #if the goal of the chart is to analyze the monthly variation in tactics rather than segments 
        #we can use the except statement
        
        except KeyError:
            for tactic in dataframe['TACTIC'].unique():
                #pull the losses from the months of interest
                last_months_data = dataframe.loc[(dataframe['CHARGEOFF_MONTH'] == last_month) & (dataframe['TACTIC'] == segment)][metric].reset_index()
                two_months_data = dataframe.loc[(dataframe['CHARGEOFF_MONTH'] == two_months_ago) & (dataframe['TACTIC']==segment)][metric].reset_index()
            
            #calculate Month Over Month Change
            numerator = (two_months_data[metric] - last_months_data[metric]).iloc[0]
            denominator = two_months_data[metric].iloc[0]
            
            #prevent Div / 0 errors
            if denominator == 0:
                monthly_diff = 0.0
            else:
                monthly_diff = numerator/denominator
            #change the decimal to a percentage, rounded to two decimals
            monthly_diff *= 100
            
            try:
                monthly_diff = round(monthly_diff,2)
            except TypeError:
                monthly_diff = float(monthly_diff)
                monthly_diff = np.round(monthly_diff,2)
            
            #take the rounded number into a consistent string output, depending on a negative/positive monthly difference
            
            if str(monthly_diff)[0] == '-':
                monthly_diff = str(monthly_diff )[:5] + ' %'
            else:
                monthly_diff = str(monthly_diff)[:4] + ' %'
            
            #append that particular value into the list and iterate over all segments
            tactic_list.append(tactic)
            value_list.append(monthly_diff)
            
            #create a dataframe that combines the lists into a tabular, sensible format
            final_df = pd.DataFrame('TACTIC':tactic_list)
            final_df['monthly_difference'] = pd.DataFrame(value_list)
            final_df = final_df.transpose()
            final_df.insert(0, 'Metric: ' + metric, ['Tactic', 'Monthly % Difference'])
            return(final_df)
        